In [1]:
from google.colab import auth
auth.authenticate_user()

In [54]:
project_id = 'httparchive'

In [18]:
#| label: jun2023_top_dns_query
%%bigquery jun2023_total --project {project_id}
WITH pages AS (
  SELECT
    NET.REG_DOMAIN(SPLIT(JSON_VALUE_ARRAY(payload, "$._origin_dns.soa")[SAFE_OFFSET(0)], ". ")[SAFE_OFFSET(1)]) AS rname,
    NET.REG_DOMAIN(page) AS domain,
  FROM `all.pages`
  WHERE date = "2023-06-01"
    AND client = "mobile"
    --AND rank <= 5000
    AND is_root_page
  GROUP BY rname, domain
),
rname_ranked AS (
  SELECT
    rname
  FROM pages
  GROUP BY rname
  ORDER BY COUNT(DISTINCT domain) DESC
  LIMIT 200
)

SELECT
  COALESCE(rname_ranked.rname, 'other') AS rname,
  COUNT(DISTINCT domain) AS domains_cnt
FROM pages
LEFT JOIN rname_ranked
USING (rname)
GROUP BY rname_ranked.rname
ORDER BY domains_cnt DESC;

Query is running:   0%|          |

Downloading:   0%|          |

In [71]:
#| label: jun2023_top_dns_chart
#| fig-cap: Top Nameservers in June 2023
#| cap-location: margin
import plotly.express as px
import pandas as pd
import plotly.io as pio

pio.renderers.default = "notebook_connected"
pio.templates.default = "plotly_white"

df = jun2023_total.copy()
df = df[df['rname'].notnull()]

# Keep top DNS administrators names
df['rname'] = df['rname'].where(
    df['rname'].isin(
        df.groupby('rname')['domains_cnt'].sum().nlargest(15).index
    ), 'other')
df = df.groupby('rname').sum().reset_index()
df = df.sort_values(by='domains_cnt', ascending=False)

fig = px.pie(
    df,
    values='domains_cnt',
    names='rname',
    hole=0.1,
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [33]:
#| label: dec2023_google_dns_diff_query
%%bigquery dec2023_google_dns_diff --project {project_id}
WITH pages AS (
  SELECT
    date,
    NET.REG_DOMAIN(SPLIT(JSON_VALUE_ARRAY(payload, "$._origin_dns.soa")[SAFE_OFFSET(0)], ". ")[SAFE_OFFSET(1)]) AS rname,
    NET.REG_DOMAIN(page) AS domain
  FROM `all.pages`
  WHERE date IN ("2023-06-01", "2023-12-01")
    AND client = "mobile"
    AND is_root_page
    --AND rank <= 100000
  GROUP BY date, domain, rname
), jun2023 AS (
  SELECT
    rname,
    domain
  FROM pages
  WHERE date = "2023-06-01"
    AND rname = "google.com"
), dec2023 AS (
  SELECT
    domain,
    rname
  FROM pages
  WHERE date = "2023-12-01"
    AND domain IN (SELECT DISTINCT domain FROM jun2023)
), dec2023_rname_ranked AS (
  SELECT
    rname
  FROM dec2023
  GROUP BY rname
  ORDER BY COUNT(DISTINCT domain) DESC
  LIMIT 200
)

SELECT
  jun2023.rname AS jun2023_rname,
  COUNT(DISTINCT jun2023.domain) AS jun2023_domains_cnt,
  COALESCE(dec2023_rname_ranked.rname, 'other') AS dec2023_rname,
  COUNT(DISTINCT dec2023.domain) AS dec2023_domains_cnt,
  --STRING_AGG(DISTINCT dec2023.domain LIMIT 3) AS domain_examples
FROM jun2023
INNER JOIN dec2023
USING(domain)

LEFT JOIN dec2023_rname_ranked
ON dec2023.rname = dec2023_rname_ranked.rname

GROUP BY 1,3
ORDER BY 2 DESC;

Query is running:   0%|          |

Downloading:   0%|          |

In [68]:
#| label: dec2023_google_dns_diff_chart
#| fig-cap: "Domain Migration from Google DNS between June and December 2023"
#| cap-location: margin
import plotly.graph_objects as go
import plotly.colors as colors

# Prepare data for Sankey chart
df1 = dec2023_google_dns_diff.copy()

# Keep top DNS administrators names
df1['dec2023_rname'] = df1['dec2023_rname'].where(
    df1['dec2023_rname'].isin(
        df1.groupby('dec2023_rname')['dec2023_domains_cnt'].sum().nlargest(20).index
    ), 'other')
df1 = df1.groupby(['jun2023_rname', 'dec2023_rname']).sum().reset_index()
df1 = df1.sort_values(by='jun2023_domains_cnt', ascending=False)

df1['dec2023_rname'] = df1['dec2023_rname'] + ' '

source = []
target = []
value = []
labels = []
label_map = {}

for index, row in df1.iterrows():
  initial_ns = row['jun2023_rname']
  dec2023_ns = row['dec2023_rname']
  initial_domains_cnt = row['jun2023_domains_cnt']

  if initial_ns not in label_map:
    label_map[initial_ns] = len(labels)
    labels.append(initial_ns)
  if dec2023_ns not in label_map:
    label_map[dec2023_ns] = len(labels)
    labels.append(dec2023_ns)

  source.append(label_map[initial_ns])
  target.append(label_map[dec2023_ns])
  value.append(initial_domains_cnt)

# Create Sankey chart
fig = go.Figure(
    data=[
        go.Sankey(
            node = dict(
                pad = 15,
                thickness = 20,
                line = dict(color = "black", width = 0.5),
                label = labels,
            ),
            link = dict(
                source = source,
                target = target,
                value = value,
            ),
            arrangement = 'freeform'
        )
    ]
)

fig.show()

In [36]:
#| label: dec2023_all_dns_diff_query
%%bigquery dec2023_all_dns_diff --project {project_id}
WITH pages AS (
  SELECT
    date,
    NET.REG_DOMAIN(SPLIT(JSON_VALUE_ARRAY(payload, "$._origin_dns.soa")[SAFE_OFFSET(0)], ". ")[SAFE_OFFSET(1)]) AS rname,
    NET.host(page) AS domain
  FROM `all.pages`
  WHERE date IN ("2023-06-01", "2023-12-01")
    AND client = "mobile"
    AND is_root_page
    --AND rank <= 10000
  GROUP BY date, domain, rname
), jun2023 AS (
  SELECT
    rname,
    domain
  FROM pages
  WHERE date = "2023-06-01"
), dec2023 AS (
  SELECT
    domain,
    rname
  FROM pages
  WHERE date = "2023-12-01"
    AND domain IN (SELECT DISTINCT domain FROM jun2023)
), jun2023_rname_ranked AS (
  SELECT
    rname
  FROM jun2023
  GROUP BY rname
  ORDER BY COUNT(DISTINCT domain) DESC
  LIMIT 200
), dec2023_rname_ranked AS (
  SELECT
    rname
  FROM dec2023
  GROUP BY rname
  ORDER BY COUNT(DISTINCT domain) DESC
  LIMIT 200
)

SELECT
  COALESCE(jun2023_rname_ranked.rname, 'other') AS jun2023_rname,
  COUNT(DISTINCT jun2023.domain) AS jun2023_domains_cnt,
  COALESCE(dec2023_rname_ranked.rname, 'other') AS dec2023_rname,
  COUNT(DISTINCT dec2023.domain) AS dec2023_domains_cnt
FROM jun2023
INNER JOIN dec2023
USING(domain)

LEFT JOIN jun2023_rname_ranked
ON jun2023.rname = jun2023_rname_ranked.rname
LEFT JOIN dec2023_rname_ranked
ON dec2023.rname = dec2023_rname_ranked.rname

GROUP BY 1,3
ORDER BY 2 DESC;

Query is running:   0%|          |

Downloading:   0%|          |

In [70]:
#| label: dec2023_all_dns_diff_chart
#| fig-cap: "Domain Migration among DNS Services between June and December 2023"
#| cap-location: margin
import plotly.graph_objects as go
import plotly.colors as colors

df2 = dec2023_all_dns_diff.copy()

# keep only migrated domains
#df2 = df2[df2['jun2023_rname'] != df2['dec2023_rname']]

# Keep top DNS administrators names
df2['jun2023_rname'] = df2['jun2023_rname'].where(
    df2['jun2023_rname'].isin(
        df2.groupby('jun2023_rname')['jun2023_domains_cnt'].sum().nlargest(20).index
    ), 'other')
df2['dec2023_rname'] = df2['dec2023_rname'].where(
    df2['dec2023_rname'].isin(
        df2.groupby('dec2023_rname')['dec2023_domains_cnt'].sum().nlargest(20).index
    ), 'other')
df2 = df2.groupby(['jun2023_rname', 'dec2023_rname']).sum().reset_index()

df2 = df2.sort_values(by='jun2023_domains_cnt', ascending=False)
df2['dec2023_rname'] = df2['dec2023_rname'] + ' '

source = []
target = []
value = []
labels = []
label_map = {}

for index, row in df2.iterrows():
  initial_ns = row['jun2023_rname']
  dec2023_ns = row['dec2023_rname']
  initial_domains_cnt = row['jun2023_domains_cnt']

  if initial_ns not in label_map:
    label_map[initial_ns] = len(labels)
    labels.append(initial_ns)
  if dec2023_ns not in label_map:
    label_map[dec2023_ns] = len(labels)
    labels.append(dec2023_ns)

  source.append(label_map[initial_ns])
  target.append(label_map[dec2023_ns])
  value.append(initial_domains_cnt)

fig = go.Figure(
    data=[
        go.Sankey(
            node = dict(
                pad = 15,
                thickness = 20,
                line = dict(color = "black", width = 0.5),
                label = labels,
            ),
            link = dict(
                source = source,
                target = target,
                value = value,
            ),
            arrangement = 'freeform'
        )
    ]
)

fig.show()

In [40]:
#| label: sep2024_google_dns_diff_query
%%bigquery sep2024_google_dns_diff --project {project_id}
WITH pages AS (
  SELECT
    date,
    NET.REG_DOMAIN(SPLIT(JSON_VALUE_ARRAY(payload, "$._origin_dns.soa")[SAFE_OFFSET(0)], ". ")[SAFE_OFFSET(1)]) AS rname,
    NET.REG_DOMAIN(page) AS domain
  FROM `all.pages`
  WHERE date IN ("2023-06-01", "2024-09-01")
    AND client = "mobile"
    AND is_root_page
    --AND rank <= 100000
  GROUP BY date, domain, rname
), jun2023 AS (
  SELECT
    rname,
    domain
  FROM pages
  WHERE date = "2023-06-01"
    AND rname = "google.com"
), sep2024 AS (
  SELECT
    domain,
    rname
  FROM pages
  WHERE date = "2024-09-01"
    AND domain IN (SELECT DISTINCT domain FROM jun2023)
), sep2024_rname_ranked AS (
  SELECT
    rname
  FROM sep2024
  GROUP BY rname
  ORDER BY COUNT(DISTINCT domain) DESC
  LIMIT 200
)

SELECT
  jun2023.rname AS jun2023_rname,
  COUNT(DISTINCT jun2023.domain) AS jun2023_domains_cnt,
  COALESCE(sep2024_rname_ranked.rname, 'other') AS sep2024_rname,
  COUNT(DISTINCT sep2024.domain) AS sep2024_domains_cnt
FROM jun2023
INNER JOIN sep2024
USING(domain)

LEFT JOIN sep2024_rname_ranked
ON sep2024.rname = sep2024_rname_ranked.rname

GROUP BY 1,3
ORDER BY 2 DESC;

Query is running:   0%|          |

Downloading:   0%|          |

In [72]:
#| label: sep2024_google_dns_diff_chart
#| fig-cap: "Domain Migration from Google DNS between June 2023 and September 2024"
#| cap-location: margin
import plotly.graph_objects as go
import plotly.colors as colors

# Prepare data for Sankey chart
df1 = sep2024_google_dns_diff.copy()

# Keep top DNS administrators names
df1['sep2024_rname'] = df1['sep2024_rname'].where(
    df1['sep2024_rname'].isin(
        df1.groupby('sep2024_rname')['sep2024_domains_cnt'].sum().nlargest(20).index
    ), 'other')
df1 = df1.groupby(['jun2023_rname', 'sep2024_rname']).sum().reset_index()
df1 = df1.sort_values(by='jun2023_domains_cnt', ascending=False)

df1['sep2024_rname'] = df1['sep2024_rname'] + ' '

source = []
target = []
value = []
labels = []
label_map = {}

for index, row in df1.iterrows():
  initial_ns = row['jun2023_rname']
  sep2024_ns = row['sep2024_rname']
  initial_domains_cnt = row['jun2023_domains_cnt']

  if initial_ns not in label_map:
    label_map[initial_ns] = len(labels)
    labels.append(initial_ns)
  if sep2024_ns not in label_map:
    label_map[sep2024_ns] = len(labels)
    labels.append(sep2024_ns)

  source.append(label_map[initial_ns])
  target.append(label_map[sep2024_ns])
  value.append(initial_domains_cnt)

# Create Sankey chart
fig = go.Figure(
    data=[
        go.Sankey(
            node = dict(
                pad = 15,
                thickness = 20,
                line = dict(color = "black", width = 0.5),
                label = labels,
            ),
            link = dict(
                source = source,
                target = target,
                value = value,
            ),
            arrangement = 'freeform'
        )
    ]
)

fig.show()